In [1]:
import openpyxl as oxl

from histopath_bim_des.config import Config
from histopath_bim_des.model import Model
from histopath_bim_des.process.core import Process

In [2]:
path = '../../assets/config_base.xlsx'
wbook = oxl.load_workbook(path, data_only=True)
cfg = Config.from_workbook(wbook, sim_hours=10*168, num_reps=10)

In [3]:
model = Model(cfg)
model

Model (default environment)

## Model processes

In [4]:
model.processes

{'arrive_reception': Process (arrive_reception),
 'booking_in': Process (booking_in),
 'batcher.reception_to_cutup': BatchingProcess (batcher.reception_to_cutup),
 'reception_to_cutup': DeliveryProcess (deliveryprocess.0),
 'cutup_start': Process (cutup_start),
 'cutup_bms': Process (cutup_bms),
 'cutup_pool': Process (cutup_pool),
 'cutup_large': Process (cutup_large),
 'batcher.cutup_bms_to_processing': BatchingProcess (batcher.cutup_bms_to_processing),
 'cutup_bms_to_processing': DeliveryProcess (cutup_bms_to_processing),
 'batcher.cutup_pool_to_processing': BatchingProcess (batcher.cutup_pool_to_processing),
 'cutup_pool_to_processing': DeliveryProcess (cutup_pool_to_processing),
 'batcher.cutup_large_to_processing': BatchingProcess (batcher.cutup_large_to_processing),
 'cutup_large_to_processing': DeliveryProcess (cutup_large_to_processing),
 'processing_start': Process (processing_start),
 'decalc_bone_station': Process (decalc_bone_station),
 'decalc_oven': Process (decalc_oven)

### Input types

In [5]:
{
    k: str(v.in_type).removeprefix("<class '").removesuffix("'>")
    .replace('histopath_bim_des.specimens.','')
    for k, v in model.processes.items() if isinstance(v, Process)
}

{'arrive_reception': 'Specimen',
 'booking_in': 'Specimen',
 'cutup_start': 'Specimen',
 'cutup_bms': 'Specimen',
 'cutup_pool': 'Specimen',
 'cutup_large': 'Specimen',
 'processing_start': 'Specimen',
 'decalc_bone_station': 'Batch[Block]',
 'decalc_oven': 'Block',
 'processing_assign_queue': 'Block',
 'processing_urgents': 'Batch[Block]',
 'processing_smalls': 'Batch[Block]',
 'processing_larges': 'Batch[Block]',
 'processing_megas': 'Batch[Block]',
 'embed_and_trim': 'Block',
 'post_processing': 'Specimen',
 'microtomy': 'Specimen',
 'staining_start': 'Specimen',
 'staining_regular': 'Batch[Slide]',
 'staining_megas': 'Batch[Slide]',
 'post_staining': 'Specimen',
 'labelling': 'Specimen',
 'scanning_start': 'Specimen',
 'scanning_regular': 'Batch[Slide]',
 'scanning_megas': 'Batch[Slide]',
 'post_scanning': 'Specimen',
 'qc': 'Specimen',
 'assign_histopath': 'Specimen',
 'report': 'Specimen'}

### Model execution and KPIs

In [6]:
model.run()

In [7]:
{k: v() for k,v in model.wips.__dict__.items()}

{'total': 3024,
 'in_reception': 15,
 'in_cut_up': 51,
 'in_processing': 1073,
 'in_microtomy': 0,
 'in_staining': 622,
 'in_labelling': 24,
 'in_scanning': 297,
 'in_qc': 143,
 'in_reporting': 189}

In [8]:
{k: v.requesters().length() for k,v in model.resources.__dict__.items()}

{'booking_in_staff': 15,
 'bms': 45,
 'cut_up_assistant': 26,
 'processing_room_staff': 11,
 'microtomy_staff': 25,
 'staining_staff': 40,
 'scanning_staff': 172,
 'qc_staff': 532,
 'histopathologist': 189,
 'bone_station': 0,
 'processing_machine': 6,
 'staining_machine': 38,
 'coverslip_machine': 0,
 'scanning_machine_regular': 0,
 'scanning_machine_megas': 169}

In [9]:
tat_dict = {k:v['report_end']-v['reception_start'] for k,v in model.specimen_data.items() if 'report_end' in v}

In [10]:
import numpy as np

ar = np.array(list(tat_dict.values()))
print(np.min(ar), np.median(ar), np.mean(ar), np.max(ar))

47.412639944936075 234.574761938836 243.59276827765328 623.9005412065112


In [11]:
print(np.mean(ar < model.days(7)))
print(np.mean(ar < model.days(10)))
print(np.mean(ar < model.days(12)))
print(np.mean(ar < model.days(24)))

0.12870362509550895
0.5212666610068767
0.7481110450802275
0.9998302062993463
